# Case Studies

## Abstract Class

ReactEA Case Study Abstract Class.

Child Classes must implement all abstract methods (`objective`, `name` and `feval_names`).

In [1]:
from abc import abstractmethod, ABC


class CaseStudy(ABC):
    """
    Base class for all Case Studies.
    A Case Study defines the evaluation functions to use in the optimization problem.
    """

    def __init__(self, multi_objective: bool = False):
        """
        Initializes the case study at a class level.

        Parameters
        ----------
        multi_objective: bool
            boolean defining if we are facing a single or multi-objective optimization problem.
        """
        self.multi_objective = multi_objective

    @abstractmethod
    def objective(self):
        """
        Defines the evaluation functions to use in the optimization problem taking into account if we are facing a
        single or multi-objective problem.

        Returns
        -------
        Problem
            Problem object defining the evaluation functions of the optimization problem.
        """
        raise NotImplementedError

    @abstractmethod
    def name(self):
        """
        Defines the name of the Case Study.

        Returns
        -------
        str
            Name of the case study.
        """
        return NotImplementedError

    @abstractmethod
    def feval_names(self):
        """
        Defines the names of the evaluation functions used in the Case Study.

        Returns
        -------
        str
            Name of the evaluation functions used in the case study.
        """
        return NotImplementedError

## Example of how to implement you own case studies

It is easy to implement your own case studies.

In ReactEA case studies can be used to optimize one or many evaluation functions.

### Using the evaluation function wrapper:

You need to provide:
   - one or a list of evaluation functions
  - whether to use multi-objective or single-objective (many evaluations can be aggregated into one);
  - the name of the case study;
  - the weights of each evaluation function (optional, used in the case you provide multiple evaluation functions but want to use a single-objective EA where the objectives are aggregated and wheighted based on this parameter).

### First you need to specify the evaluation functions to optimize (see [evaluation_functions.ipynb](evaluation_functions.ipynb) for more details).

In [5]:
from rdkit.Chem.QED import qed
from reactea import evaluation_functions_wrapper

# evaluation function returning the number of rings a molecule
def number_of_rings(mol):
    ri = mol.GetRingInfo()
    n_rings = len(ri.AtomRings())
    return n_rings

n_rigs_feval = evaluation_functions_wrapper(number_of_rings, maximize=False, worst_fitness=100, name='n_rings')

# evaluation function returning the drug-likeliness score (QED) of a molecule
def qed_score(mol):
    return qed(mol)

qed_feval = evaluation_functions_wrapper(qed_score, maximize=True, worst_fitness=0.0, name='qed')

### Now the case study:

In [6]:
from reactea import case_study_wrapper

# case study to optimize a single objective `f1` (minimize number of rings in a molecule)
minimize_rings = case_study_wrapper(n_rigs_feval, multi_objective=False, name='minimize_rings')

# case study to optimize a single objective but with multiple evaluation functions `f1` and `f2` (minimize number of rings in a molecule and maximize qed)
# the number of evaluation functions must be the same as the number of values in weights and the sum of the weights must be 1
minimize_rings_maximize_qed = case_study_wrapper([n_rigs_feval, qed_feval], multi_objective=False, name='minimize_rings_maximize_qed', weights=[0.3, 0.7])

# case study to optimize multiple objectives simultaneous
minimize_rings_maximize_qed_mo = case_study_wrapper([n_rigs_feval, qed_feval], multi_objective=True, name='minimize_rings_maximize_qed_mo')

## Creating your own class:

The class needs to inherit from the `CaseStudt` class.
It needs to implement its abstract methods (`objective`, `name` and `feval_names`).

In [7]:
from reactea.optimization.evaluation import AggregatedSum
from reactea.optimization.problem import ChemicalProblem

# single objective Case Study
class RingsCaseStudy(CaseStudy):

    def __init__(self):
        super(RingsCaseStudy, self).__init__(multi_objective=False)
        self.feval_names_str = None

    def objective(self):
        # define evaluation functions
        f1 = n_rigs_feval()
        problem = ChemicalProblem([f1])
        self.feval_names_str = f"{f1.method_str()}"
        return problem

    def name(self):
        return f"RingsCaseStudy"

    def feval_names(self):
        return self.feval_names_str

# case study that can be used to single and multi-objective
class RingsQedCaseStudy(CaseStudy):

    def __init__(self, multi_objective):
        super(RingsQedCaseStudy, self).__init__(multi_objective=multi_objective)
        self.multi_objective = multi_objective
        self.feval_names_str = None

    def objective(self):
        # define evaluation functions
        f1 = n_rigs_feval()
        f2 = qed_feval()

        if self.multi_objective:
            problem = ChemicalProblem([f1, f2])
            self.feval_names_str = f"{f1.method_str()};{f2.method_str()}"
            return problem
        else:
            # in case of single-objective with many evaluation functions use AggregatedSum and provide weights
            f_ag = AggregatedSum([f1, f2], [0.25, 0.25, 0.5])
            problem = ChemicalProblem([f_ag])
            self.feval_names_str = f"{f_ag.method_str()}"
            return problem

    def name(self):
        return f"RingsQedCaseStudy"

    def feval_names(self):
        return self.feval_names_str